# ANALISIS DEL PROCESADO DE MENA DE HIERRO EN UNA PLANTA DE FLOTACION MINERA

## Preprocessing

Importar el dataset:

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
#from scipy.stats import pearsonr, spearmanr
%load_ext autoreload
%autoreload 2
import utils
import datetime

df = pd.read_csv("res/MiningProcess_Flotation_Plant_Database.csv")

In [ ]:
df.dtypes

In [ ]:
df.isna().any()

Hacer formateo de tipos sobre cada columna

In [ ]:
df["date"] = pd.to_datetime(df["date"])
#decimal was represented by commas, change to point
columns_to_float = [
    "% Iron Feed", "% Silica Feed", "Starch Flow", 
    "Amina Flow", "Ore Pulp Flow", "Ore Pulp pH", "Ore Pulp Density",
    'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow',
    'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow',
    'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow',
    'Flotation Column 07 Air Flow', 'Flotation Column 01 Level',
    'Flotation Column 02 Level', 'Flotation Column 03 Level',
    'Flotation Column 04 Level', 'Flotation Column 05 Level',
    'Flotation Column 06 Level', 'Flotation Column 07 Level',
    '% Iron Concentrate', '% Silica Concentrate'
]
for col in columns_to_float:
    df[col] = df[col].str.replace(',', '.')
    df[col] = df[col].astype(np.float64)

In [ ]:
df.dtypes

Hay varias columnas para indicar las lecturas de los sensores sobre el flujo de aire y el nivel de las celdas de flotación. Se podrían juntar todas en una utilizando la media de las lecturas de los sensores?

In [ ]:
airflow_cols = df.filter(regex='.*Air Flow$').columns
level_cols = df.filter(regex='.*Level$').columns

df["airflow"] = df[airflow_cols].mean(axis=1)
df["level"] = df[level_cols].mean(axis=1)

df = df.drop(np.concatenate([airflow_cols.tolist(), level_cols.tolist()]).tolist(), axis=1)

diccionario de número de mes a nombre de mes:

In [ ]:
month_number_to_name = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}

In [ ]:
df['month'] = ((df['date'].dt.month).map(month_number_to_name)).astype('object')
df['week'] = (df['date'].dt.isocalendar().week).astype('object')
df['day'] = (df['date'].dt.day).astype('object')

In [ ]:
df.dtypes

Creamos una versión del dataset sólo con las columnas nuḿericas, sin fechas

In [ ]:
df_numeric = df.select_dtypes(np.float64)

Comprobamos que todas las columnas numéricas tienen valores positivos

In [ ]:
df_numeric.apply(lambda x: (x > 0).all())

## EDA (Análisis Exploratorio de Datos)

### Análisis general

Vemos el dataset limpio:

In [ ]:
df.info()

In [ ]:
df_numeric.describe()

Comprobamos que todas las columnas tienen valores positivos...

In [ ]:
df_numeric.corr()

In [ ]:
#sns.heatmap(df_nodates, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=.5, cbar_kws={"shrink": .5})

### Análisis de estacionalidad

por mes

In [ ]:
utils.custom_boxplot(df, "month", "% Silica Concentrate", "Visualización de la cantidad de silicio por mes")

por semana

In [ ]:
utils.custom_boxplot(df, "week", "% Silica Concentrate", "Visualización de la cantidad de silicio por semana")

media de días dentro de mes

In [ ]:
utils.custom_boxplot(df, "day", "% Silica Concentrate", "Visualización de la cantidad de silicio por cada día de los meses")

Se puede ver cierta estacionalidad, en los últimos días de cada semana de un mes incrementa el silicio.